<a href="https://colab.research.google.com/github/aldolipani/SDAE22/blob/master/SDAE_DL_Tutorial_on_Lucas_et_al.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 3 - Geospatial Engineering
## Predicting Electrical Conductivity with Deep Learning

### Aim
In this tutorial we will see how to solve this task using tabular data, raster data, and their combination.


### Acknowledgement
This notebook is written by [Dr. Aldo Lipani](aldolipani.com)

Install the required packages:

In [ ]:
! pip install rasterio livelossplot torchviz

And import all we need:

In [ ]:
from glob import glob

import numpy as np
import pandas as pd
import rasterio
import torch
import torch.optim as optim
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot
from matplotlib import pyplot as plot
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchviz import make_dot
from tqdm import tqdm

## Load the Tabular Dataset

Let's load the dataset and remove any Not a Numbers (NAs) it may contain:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/aldolipani/SDAE22/master/data/logEC_2015_wgs_soil_cov.csv', index_col=0)

df.dropna(inplace=True)
df.head()

In [ ]:
print(f'Number of samples: {len(df)}')

To make the training quick, we decided to select only those points that lie within the Sicilian region (Italy).

In [ ]:
# bounding box of Sicily in WGS
bbox_sicily = {'xwgs_0': 12.221645,
               'xwgs_1': 15.682920,
               'ywgs_0': 36.304816,
               'ywgs_1': 38.582695}

df_sicily = df[(df['xwgs'] >= bbox_sicily['xwgs_0']) & (df['ywgs'] >= bbox_sicily['ywgs_0']) &
               (df['xwgs'] <= bbox_sicily['xwgs_1']) & (df['ywgs'] <= bbox_sicily['ywgs_1'])]

df_sicily.describe()

In [ ]:
print('Number of samples in Sicily:', len(df_sicily))

### Normalization

This dataset contains only continues variables. This means that we do not need to encode any categorical variables. To normalize each values we can use a Min Max normalization from the scikit-learn packages. Please note that we do the same also for the target variable. For neural networks, it is always recomended to have attributes with values that range within -3 to 3. This in order to avoid exploding gradient issues while training.

In [ ]:
# we make a copy of all rows
all_rows = df_sicily.copy()

# we copy and transform the target attribute
ys = np.array(all_rows['y']).reshape((-1, 1))
min_max_scaler_ys = MinMaxScaler()
ys = min_max_scaler_ys.fit_transform(ys)
ys = ys.reshape(-1)

# we delete the target attributes from all_rows
del all_rows['y']
del all_rows['xwgs']
del all_rows['ywgs']

# we transform the samples
min_max_scaler_xs = MinMaxScaler()
all_rows = min_max_scaler_xs.fit_transform(all_rows)

The tabular dataset is now ready.

## Load the Raster Dataset

In this tutorial we will try to combine NDVI rasters to the tabular data above. These rasters are about Sicily. In order to use the rasters, we first need to download them:

In [ ]:
! git clone https://github.com/aldolipani/SDAE22.git
! mkdir data
! mv SDAE22/data/rasters ./data/rasters
! rm -fr SDAE22

Let's now load the rasters:

In [ ]:
rasters = {}

for file in tqdm(glob('data/rasters/*.tif')):
    feature = file.split('/')[-1][:-4]
    rasters[feature] = rasterio.open(file)

print('Raster features:', list(rasters.keys()))

Let's have a look if the NDVI 7 raster looks like Sicily:

In [ ]:
plot.imshow(rasters['NDVI_M7_02'].read(1), vmin=-1, vmax=1, cmap='pink')
plot.show()

Let's now have a look at the empirical distribution of values of this raster:

In [ ]:
plot.hist(rasters['NDVI_M7_02'].read(1).reshape((-1,)), bins=50)
plot.show()

NDVI is $\in [0, 1]$. However, we can observe that there other values at 255. These values represent missing NDVI values due to sea or clouds. We will replace these values with -1s. We now visualize the empiricial distribution around the normal range of NDVI.

In [ ]:
plot.hist(rasters['NDVI_M7_02'].read(1).reshape((-1,)), range=(-1, 1), bins=50)
plot.show()

In [ ]:
plot.hist(rasters['NDVI_M7_02'].read(1).reshape((-1,)), range=(250, 256), bins=50)
plot.show()

We now clip squares around the tabular dataset points of size 10:

In [ ]:
half_window_size = 5 # change this number of if you want to have larger windows
# this list will contain all images
all_images = []
for index, row in tqdm(df_sicily.iterrows()):
    all_images.append({})
    for raster in rasters:
        x, y = rasters[raster].index(row['xwgs'], row['ywgs'])
        clip = rasters[raster].read(1)[x - half_window_size:x + half_window_size + 1, y - half_window_size + 1:y + half_window_size + 2]
        all_images[-1][raster] = np.array(clip)

The raster dataset is now ready.

# Train, Validation, Test Splits

We will perform a 8:1:1 train, validation and test splits. This will be done applying the `train_test_split` of scikit-lean twice.

In [ ]:
rows_train, rows_test, images_train, images_test, ys_train, ys_test = train_test_split(all_rows, all_images, ys,
                                                                                       test_size=0.1, random_state=42)

rows_train, rows_val, images_train, images_val, ys_train, ys_val = train_test_split(rows_train, images_train, ys_train,
                                                                                    test_size=0.111111, random_state=42)

print(f'training set size: {len(rows_train)}, validation set size: {len(rows_val)}, testing set size: {len(rows_test)}')

We now define a helper class to use the dataset. This class will take as input the tabular data (`all_rows`), the raster data (`all_images`) and labels (`ys`) and help us iterate over the dataset in order.

In [ ]:
class RowImageDataset(Dataset):

    def __init__(self, all_rows, all_images, ys):
        self.all_rows = all_rows
        self.all_images = all_images
        self.ys = ys

    def __len__(self):
        return len(self.all_rows)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        row = torch.tensor(self.all_rows[idx])
        images = self.all_images[idx]
        image = torch.stack([torch.from_numpy(images[k]) for k in sorted(images.keys())], dim=0).double()
        y = torch.tensor(self.ys[idx]).reshape((-1,))

        sample = {'row': row, 'image': image, 'y': y}
        return sample

# we initialize this dataset for the training set
dataset_train = RowImageDataset(rows_train,
                                images_train,
                                ys_train)

# the validation set
dataset_val = RowImageDataset(rows_val,
                              images_val,
                              ys_val)

# and the test set
dataset_test = RowImageDataset(rows_test,
                               images_test,
                               ys_test)

# we now output the first training example
dataset_train[0]

# Deep Learning Architectures

We will explore the predictive power of 3 deep learning architectures: one architecture for tabular data, one for raster data and one that combines the two.

## Tabular Encoder

We now define an encoder architecture for tabular data that can be also used as a regressor when we set the number of output features to 1.

In [ ]:
class RowEncoder(nn.Module):

    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 1024)
        self.act_fn = nn.Tanh()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(1024, out_features)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act_fn(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

We now draw this encoder architecture for tabular data.

In [ ]:
row_encoder = RowEncoder(rows_train.shape[1], 1).double()
sample = dataset_train[0]
output = row_encoder(sample["row"])
make_dot(output, params=dict(list(row_encoder.named_parameters())))

This model has not been trained yet but we can already use it. Here is its output:

In [ ]:
print(f'output of the untrained model {output.item()}, true value is {sample["y"].item()}')

Following we define a training procedure to train the model above.

In [ ]:
row_encoder = RowEncoder(rows_train.shape[1], 1).double()

# batch size for the training loop
batch_size = 10
loader_train = DataLoader(dataset_train,
                          batch_size=batch_size,
                          shuffle=True)
loader_val = DataLoader(dataset_val,
                        batch_size=batch_size,
                        shuffle=False)

# define the loss function
criterion = nn.MSELoss()
# define the optimizer and tell the optmize which parameter it should optmize
optimizer = optim.Adam(row_encoder.parameters(), lr=0.00005)

# used to visualize the losses
plot_losses = PlotLosses(outputs=[MatplotlibPlot()])

# early stopping configuration
min_validation_loss = float('inf')
max_patience = 20
patience = max_patience

for epoch in range(1000):  # loop over the dataset multiple times
    running_loss = 0.0
    n_loss = 0
    validation_loss = 0.0
    for i, batch in enumerate(loader_train, 0):
        # unpack the batch
        rows, images, ys = batch['row'], batch['image'], batch['y']
        # set the model in training mode
        row_encoder.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward step
        outputs = row_encoder(rows)
        # compute the loss
        loss = criterion(outputs, ys)
        # backpropagation step
        loss.backward()
        optimizer.step()

        # compute losses and show them
        running_loss += loss.item()
        n_loss += len(batch)
        if i % 5 == 4:
            row_encoder.eval()
            validation_loss = 0.0
            for batch in loader_val:
                rows, images, ys = batch['row'], batch['image'], batch['y']
                outputs = row_encoder(rows)
                validation_loss += criterion(outputs, ys).item()
            plot_losses.update({'loss': running_loss / n_loss, 'val_loss': validation_loss / len(dataset_val)})
            plot_losses.send()  # update plots
            running_loss = 0.0
            n_loss = 0

    # early stopping checks
    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        # save model or overwrite it if the validation loss has improved
        torch.save(row_encoder.state_dict(), 'row_encoder_weights.pth')
        patience = max_patience
    else:
        patience -= 1

    if patience == 0:
        print('Early stopping triggered at epoch', epoch)
        break

print(f'Training finished with best validation loss: {min_validation_loss}')

Let's now print the output of the trained model:

In [ ]:
print(f'output of the untrained model {row_encoder(sample["row"]).item()}, true value is {sample["y"].item()}')

## Raster Encoder

We now define an encoder architecture for raster data that can be also used as a regressor when we set the number of output features to 1. Note that while the previous was a Multi-Layer Perceptron based architecture, this will be a Convolutional Neural Network.

In [ ]:
class ImageEncoder(nn.Module):

    def __init__(self, num_rasters, out_features=1):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=num_rasters,
            out_channels=20,
            kernel_size=3,
            stride=1)
        self.dropout2d1 = nn.Dropout2d(0.2)
        self.conv2 = nn.Conv2d(
            in_channels=20,
            out_channels=10,
            kernel_size=3,
            stride=1)
        self.dropout2d2 = nn.Dropout2d(0.2)
        self.act_fn = nn.Tanh()
        self.fc1 = nn.Linear(490, out_features)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.act_fn(x)
        x = self.dropout2d1(x)
        x = self.conv2(x)
        x = self.act_fn(x)
        x = self.dropout2d2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        return x

We now draw this encoder architecture for raster data.

In [ ]:
image_encoder = ImageEncoder(sample['image'].shape[0], 1).double()
sample = dataset_train[0]
output = image_encoder(sample['image'].reshape(1, -1, 11, 11))
make_dot(output, params=dict(list(image_encoder.named_parameters())))

We now print the output of the untrained model.

In [ ]:
print(f'output of the untrained model {output.item()}, true value is {sample["y"].item()}')

We perform the sample training as done for the encoder for tabular data.

In [ ]:
image_encoder = ImageEncoder(sample['image'].shape[0], 1).double()

batch_size = 10
loader_train = DataLoader(dataset_train,
                          batch_size=batch_size,
                          shuffle=True)

loader_val = DataLoader(dataset_val,
                        batch_size=batch_size,
                        shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(image_encoder.parameters(), lr=0.00005)

plot_losses = PlotLosses(outputs=[MatplotlibPlot()])

min_validation_loss = float('inf')
max_patience = 10
patience = max_patience

for epoch in range(1000):  # loop over the dataset multiple times
    running_loss = 0.0
    validation_loss = 0.0
    n_loss = 0
    for i, batch in enumerate(loader_train, 0):
        rows, images, ys = batch['row'], batch['image'], batch['y']

        image_encoder.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = image_encoder(images)
        loss = criterion(outputs, ys)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        n_loss += len(batch)
        if i % 5 == 4:
            image_encoder.eval()
            validation_loss = 0.0
            for batch in loader_val:
                rows, images, ys = batch['row'], batch['image'], batch['y']
                outputs = image_encoder(images)
                validation_loss += criterion(outputs, ys).item()
            plot_losses.update({'loss': running_loss / n_loss, 'val_loss': validation_loss / len(dataset_val)})
            plot_losses.send()  # draw, update logs, etc
            running_loss = 0.0
            n_loss = 0

    # early stopping strategy
    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        torch.save(image_encoder.state_dict(), 'image_encoder_weights.pth')
        patience = max_patience
    else:
        patience -= 1

    if patience == 0:
        print('Early stopping triggered at epoch', epoch)
        break

print(f'Training finished with best validation loss: {min_validation_loss}')

Let's now print the output of the trained model:

In [ ]:
print(f'output of the untrained model {image_encoder(sample["image"].reshape(1, -1, 11, 11)).item()}, true value is {sample["y"].item()}')

## Combine Architecture

We now define a combine architecture that is able to handle tabular and raster data using the two encoders defined above. Note that we are using the encoders defined above using a number of output features larger than 1.

In [ ]:
class RowImageNet(nn.Module):

    def __init__(self, in_features, num_rasters):
        super().__init__()
        # encoders with output features larger than 1, we call their output embeddings
        self.row_encoder = RowEncoder(in_features, 20)
        self.image_encoder = ImageEncoder(num_rasters, 20)
        self.act_fn = nn.Tanh()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(40, 1)

    def forward(self, x_row, x_image):
        # Perform the calculation of the model to determine the prediction
        x_row = self.row_encoder(x_row)
        x_image = self.image_encoder(x_image)
        # concatenate the output of the two encoders
        x = torch.cat((x_row, x_image), dim=1)
        x = self.act_fn(x)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

We now draw the combined architecture.

In [ ]:
row_image_net = RowImageNet(rows_train.shape[1], dataset_train[0]['image'].shape[0]).double()
sample = dataset_train[0]
output = row_image_net(sample['row'].reshape(1, -1), sample['image'].reshape(1, -1, 11, 11))
make_dot(output, params=dict(list(row_image_net.named_parameters())))

This model has not been trained yet but we can already use it. Here is its output:

In [ ]:
print(f'output of the untrained model {output.item()}, true value is {sample["y"].item()}')


In [ ]:
row_image_net = RowImageNet(rows_train.shape[1], sample['image'].shape[0]).double()

batch_size = 10
loader_train = DataLoader(dataset_train,
                          batch_size=batch_size,
                          shuffle=True)

loader_val = DataLoader(dataset_val,
                        batch_size=batch_size,
                        shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(row_image_net.parameters(), lr=0.00005)

plot_losses = PlotLosses(outputs=[MatplotlibPlot()])

min_validation_loss = float('inf')
max_patience = 10
patience = max_patience

for epoch in range(1000):  # loop over the dataset multiple times
    running_loss = 0.0
    n_loss = 0
    validation_loss = 0.0
    for i, batch in enumerate(loader_train, 0):
        rows, images, ys = batch['row'], batch['image'], batch['y']

        row_image_net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward step
        outputs = row_image_net(rows, images)
        # compute the loss
        loss = criterion(outputs, ys)
        # backpropagation step
        loss.backward()
        optimizer.step()

        # compute losses and show them
        running_loss += loss.item()
        n_loss += len(batch)
        if i % 5 == 4:
            row_image_net.eval()
            validation_loss = 0.0
            for i, batch in enumerate(loader_val):
                rows, images, ys = batch['row'], batch['image'], batch['y']
                outputs = row_image_net(rows, images)
                validation_loss += criterion(outputs, ys).item()

            plot_losses.update({'loss': running_loss / n_loss, 'val_loss': validation_loss / len(dataset_val)})
            plot_losses.send()  # draw, update logs, etc
            running_loss = 0.0
            n_loss = 0

    # early stopping strategy
    if validation_loss < min_validation_loss:
        min_validation_loss = validation_loss
        torch.save(row_image_net.state_dict(), 'row_image_net_weights.pth')
        patience = max_patience
    else:
        patience -= 1

    if patience == 0:
        print('Early stopping triggered at epoch', epoch)
        break

print(f'Training finished with best validation loss: {min_validation_loss}')

In [ ]:
print(f'output of the untrained model {row_image_net(sample["row"].reshape(1, -1), sample["image"].reshape(1, -1, 11, 11)).item()}, true value is {sample["y"].item()}')

# Evaluation and Comparison

Now that we have trained the models above using their best hyper-parameters (manually), we can evaluate and compare the models using the test set.

In [ ]:
row_encoder = RowEncoder(rows_train.shape[1], 1).double()
# load the weights of the best row encoder
row_encoder.load_state_dict(torch.load('row_encoder_weights.pth'))
row_encoder.eval()

batch_size = 10
loader_test = DataLoader(dataset_test,
                         batch_size=batch_size,
                         shuffle=False)

# collect the predictions
pred_ys = []
for i, batch in enumerate(loader_test, 0):
    rows, images, ys = batch['row'], batch['image'], batch['y']
    pred_ys.extend([x.item() for x in row_encoder(rows)])

# compute the evaluation measures
print(f'R2 {r2_score(ys_test, pred_ys)}')
# back transform and compute the evaluation measures
backtransformed_pred_ys = np.exp(min_max_scaler_ys.inverse_transform(np.array(pred_ys).reshape(-1, 1)))
backtransformed_ys = np.exp(min_max_scaler_ys.inverse_transform(ys_test.reshape(1, -1)).reshape(-1))
print(f'RMSE {np.sqrt(mean_squared_error(backtransformed_ys,backtransformed_pred_ys))}')

plot.plot(ys_test, pred_ys, 'o')
plot.axis('equal')
plot.plot([-1, 1], [-1, 1])

In [ ]:
image_encoder = ImageEncoder(sample['image'].shape[0], 1).double()
# load the weights of the best row encoder
row_encoder.load_state_dict(torch.load('row_encoder_weights.pth'))
row_encoder.eval()

batch_size = 10
loader_test = DataLoader(dataset_test,
                         batch_size=batch_size,
                         shuffle=False)

# collect the predictions
pred_ys = []
for i, batch in enumerate(loader_test, 0):
    rows, images, ys = batch['row'], batch['image'], batch['y']
    pred_ys.extend([x.item() for x in image_encoder(images)])

# compute the evaluation measures
print(f'R2 {r2_score(ys_test, pred_ys)}')
# back transform and compute the evaluation measures
backtransformed_pred_ys = np.exp(min_max_scaler_ys.inverse_transform(np.array(pred_ys).reshape(-1, 1)))
backtransformed_ys = np.exp(min_max_scaler_ys.inverse_transform(ys_test.reshape(1, -1)).reshape(-1))
print(f'RMSE {np.sqrt(mean_squared_error(backtransformed_ys,backtransformed_pred_ys))}')

plot.plot(ys_test, pred_ys, 'o')
plot.axis('equal')
plot.plot([-1, 1], [-1, 1])


In [ ]:
image_encoder = ImageEncoder(sample['image'].shape[0], 1).double()
# load the weights of the best row encoder
row_image_net.load_state_dict(torch.load('row_image_net_weights.pth'))
row_image_net.eval()

batch_size = 10
loader_test = DataLoader(dataset_test,
                         batch_size=batch_size,
                         shuffle=False)


# collect the predictions
pred_ys = []
for i, batch in enumerate(loader_test, 0):
    rows, images, ys = batch['row'], batch['image'], batch['y']
    pred_ys.extend([x.item() for x in row_image_net(rows, images)])

# compute the evaluation measures
print(f'R2 {r2_score(ys_test, pred_ys)}')
# back transform and compute the evaluation measures
backtransformed_pred_ys = np.exp(min_max_scaler_ys.inverse_transform(np.array(pred_ys).reshape(-1, 1)))
backtransformed_ys = np.exp(min_max_scaler_ys.inverse_transform(ys_test.reshape(1, -1)).reshape(-1))
print(f'RMSE {np.sqrt(mean_squared_error(backtransformed_ys,backtransformed_pred_ys))}')

plot.plot(ys_test, pred_ys, 'o')
plot.axis('equal')
plot.plot([-1, 1], [-1, 1])